In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [2]:
input_img = Input(shape=(28,28,1))

In [3]:
x = Conv2D(32,(3,3), activation='relu',padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(32,(3,3), activation='relu',padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
x = Conv2D(32,(3,3),activation='relu',padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(32,(3,3),activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1,(3,3),activation='sigmoid',padding='same')(x)

In [5]:
autoencoder = Model(input_img,decoded)
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')

In [10]:
(x_train,_),(x_test,_) = mnist.load_data()

In [11]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train),28,28,1))
x_test = np.reshape(x_test, (len(x_test),28,28,1))

In [12]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=x_test.shape)

In [13]:
x_train_noisy = np.clip(x_train_noisy, 0, 1)
x_test_noisy = np.clip(x_test_noisy, 0, 1)

In [17]:
# x_test_noisy.shape
# x_train_noisy.shape

In [18]:
autoencoder.fit(x_train_noisy, x_train,
               epochs = 100,
               batch_size = 128,
               shuffle=True,
               validation_data=(x_test_noisy,x_test))

decoder_imgs = autoencoder.predict(x_test)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 207s 3ms/step - loss: 0.1871 - val_loss: 0.1327
Epoch 2/100
60000/60000 [==============================] - 206s 3ms/step - loss: 0.1202 - val_loss: 0.1117
Epoch 3/100
47616/60000 [======================>.......] - ETA: 40s - loss: 0.1124

KeyboardInterrupt: 

In [ ]:
n = 10
plt.figure(figsize=(20,5))
for i in range(n):
    ax = plt.subplot(2,n,i)
    plt.imshow(x_test[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # reconstructed image
    ax = plt.subplot(2,n,i+n)
    plt.imshow(decoded_imgs[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()